# OSM Oslo city bike data

The official [Oslo city bike app](https://oslobysykkel.no/en) doesn't work properly without internet connectivity. Many bysykkel station locations are still available using offline mobile apps, sush as the OSM based [maps.me](https://maps.me/en/home). However, not all of the stations are included in OSM. The purpose of this project is to look at the quality of the OSM data in this regard, and to learn how to better use python tools for processing and visualising geospatial data in a simple way

This document is written as a [Jupyter notebook](https://maps.me/en/home), you may be missing out on interactive elements if you are reading this on eg. GitHub.


## Authoritative data source 

Based on the map at https://oslobysykkel.no/en/map. The map is populated with json formatted data from https://oslobysykkel.no/api/internal/stations. 

In [42]:
import requests
import json
stations = requests.get("https://oslobysykkel.no/api/internal/stations")
stations = stations.content.decode(stations.apparent_encoding)

stations = json.loads(stations)
stations
len(stations['stations'])


150

## OSM data

The data below (in the gist link) is from an overpass turbo query over Oslo

```
node
  [amenity=bicycle_rental]
  ({{bbox}});
out;

```

In [43]:
import requests
import json
osm_s = requests.get("https://gist.githubusercontent.com/jarmokivekas/b9d1cdfd006d3df26b871de28bebefe4/raw/450d1ab6f98de9307703583ea2655f29f688d09b/overpass.geojson")
osm_s = osm_s.content.decode(osm_s.apparent_encoding)
osm_s = json.loads(osm_s)
osm_s

bysykkels = []
for feature in osm_s['features']:
    if feature['properties']['network'] == 'Bysykkel' :
        bysykkels.append(feature) 
        

    
%whos

Variable        Type      Data/Info
-----------------------------------
bysykkels       list      n=102
feature         dict      n=4
json            module    <module 'json' from '/usr<...>hon3.5/json/__init__.py'>
oms_s           str       {\n  "type": "FeatureColl<...>]\n      }\n    }\n  ]\n}
osm_s           dict      n=5
r               module    <module 'requests' from '<...>es/requests/__init__.py'>
requests        module    <module 'requests' from '<...>es/requests/__init__.py'>
stations        dict      n=1
stations_json   str       {"stations":[{"id":178,"t<...>"longitude":10.734964}}]}


In [44]:
bysykkels

[{'geometry': {'coordinates': [10.6868353, 59.9076592], 'type': 'Point'},
  'id': 'node/408455943',
  'properties': {'@id': 'node/408455943',
   'amenity': 'bicycle_rental',
   'capacity': '12',
   'description': '33-Bygdøy v/Folkemuseet',
   'network': 'Bysykkel',
   'operator': 'ClearChannel',
   'ref': '33',
   'source': 'survey'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [10.7521008, 59.910089], 'type': 'Point'},
  'id': 'node/408457759',
  'properties': {'@id': 'node/408457759',
   'amenity': 'bicycle_rental',
   'capacity': '30',
   'description': '51-Havnegata S (Østbanehallen) v/Hotell Opera',
   'network': 'Bysykkel',
   'operator': 'ClearChannel',
   'ref': '51',
   'source': 'survey'},
  'type': 'Feature'},
 {'geometry': {'coordinates': [10.7517602, 59.910162], 'type': 'Point'},
  'id': 'node/408457760',
  'properties': {'@id': 'node/408457760',
   'amenity': 'bicycle_rental',
   'capacity': '30',
   'description': '50-Havnegata N (Østbanehallen)',
   'network': '